# Índice — Preprocesamiento de Datos

## Paso 1.1: Carga de los datos y entorno

1.1.1 Entorno y configuración  
1.1.2 Carga de datos  
1.1.3 Dimensiones y vista preliminar  
1.1.4 Conclusiones del EDA y pasos a seguir

## Paso 1.2: Limpieza de outliers

1.2.1 Metodología para su tratamiento    
1.2.2 Edad  
1.2.3 Ingresos  
1.2.4 Valoración winsorización para `ticket promedio`

## Paso 1.3: Multicolinealidad

1.3.1 Variables perfectamente correlacionadas    
1.3.2 Alata correlación entre `compras_online` y `compras_totales`   

## Paso 1.4: Distribuciones asimétricas

1.3.1 Transformación de variables con sesgo fuerte   

## Paso 1.5: Ingeniería de características

1.5.1 Variable binaria `tiene_pareja`  
1.5.2 Variables categóricas como ordinal   
1.5.3 Variables de interacción    
1.5.4 Sumar `adolescentes_casa` a `hijos_casa`      
1.5.5 Eliminar variable de año de nacimiento      

## Paso 1.6: Exportar el dataset con los datos procesados

## Paso 1.7: Preprocesamiento para clusterización

1.7.1 Escalar los datos   
1.7.2 PCA

In [1]:
# 1.1.1 Entorno y configuración
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)


Se habilitan pandas y numpy; las opciones amplían la salida para inspección tabular completa.

In [2]:
# 1.1.2 Carga de datos
DATA_FILE = r"data\interim\supermercado_features.csv"
df = pd.read_csv(DATA_FILE)

El dataset se lee desde el archivo CSV `supermercado_features.csv` que generamos en el EDA y queda disponible como `df`.

In [3]:
# 1.1.3 Dimensiones y vista preliminar
print("Shape:", df.shape)
print("Columnas:", len(df.columns))
print("\nPrimeras filas del dataset:")
df.head()

Shape: (1989, 49)
Columnas: 49

Primeras filas del dataset:


,anio_nacimiento,educacion,estado_civil,ingresos,hijos_casa,adolescentes_casa,fecha_cliente,recencia,gasto_vinos,gasto_frutas,gasto_carnes,gasto_pescado,gasto_dulces,gasto_oro,num_compras_oferta,num_compras_web,num_compras_catalogo,num_compras_tienda,num_visitas_web_mes,acepta_cmp3,acepta_cmp4,acepta_cmp5,acepta_cmp1,acepta_cmp2,reclama,respuesta,usuario_alta_datos,edad,antiguedad_dias,antiguedad_anios,gasto_total,gasto_promedio,prop_gasto_vinos,prop_gasto_frutas,prop_gasto_carnes,prop_gasto_pescado,prop_gasto_dulces,prop_gasto_oro,categorias_compradas,compras_totales,compras_online,compras_offline,tasa_compra_online,tasa_compra_oferta,ticket_promedio,tamano_hogar,total_dependientes,tiene_dependientes,hogar_unipersonal
0,1976,Universitaria,Casado,53359.0,1,1,2013-05-27,4,173,4,30,3,6,41,4,5,1,4,7,0,0,0,0,0,0,0,admin,49,4536,12.4,257,42.83,0.673,0.016,0.117,0.012,0.023,0.160,6,14,6,4,0.429,0.286,18.36,3,2,1,0
1,1989,Universitaria,Soltero,21474.0,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,0,1,us_direccion_2,36,4220,11.6,91,15.17,0.066,0.176,0.264,0.121,0.000,0.374,5,8,4,2,0.500,0.250,11.38,2,1,1,0
2,1986,Universitaria,Divorciado,41411.0,0,0,2013-12-07,11,37,32,38,11,3,18,1,2,1,4,6,0,0,0,0,0,0,0,us_direccion_2,39,4342,11.9,139,23.17,0.266,0.230,0.273,0.079,0.022,0.129,6,8,3,4,0.375,0.125,17.38,1,0,0,1
3,1953,Doctorado,Union_Libre,64504.0,1,2,2013-03-04,81,986,36,168,16,0,108,7,11,3,4,7,0,0,0,0,0,0,1,us_direccion_2,72,4620,12.6,1314,219.00,0.750,0.027,0.128,0.012,0.000,0.082,5,25,14,4,0.560,0.280,52.56,4,3,1,0
4,1982,Universitaria,Casado,65169.0,0,0,2014-01-14,23,1074,0,69,0,0,46,1,10,4,13,6,1,0,1,1,1,0,1,us_direccion_1,43,4304,11.8,1189,198.17,0.903,0.000,0.058,0.000,0.000,0.039,3,28,14,13,0.500,0.036,42.46,1,0,0,1




### Problemas de calidad de datos que requieren atención

1. **Errores de captura confirmados**:
   - 3 registros con edad >120 años (eliminar o imputar)
   - 1 registro con ingresos = 666,666 (valor placeholder, eliminar o imputar)

2. **Multicolinealidad detectada**:
   - `compras_totales` ↔ `compras_online` (r=0.91): Considerar eliminar una variable o crear ratio
   - `tamano_hogar` ↔ `total_dependientes` (r=1.00): Perfectamente correlacionadas, eliminar una
   - `gasto_total` con múltiples componentes (r>0.88): Normal, ya que es suma de gastos específicos

3. **Distribuciones asimétricas** (candidatas a transformación log):
   - `gasto_total`: Fuerte sesgo a la derecha
   - `ticket_promedio`: 5.4% outliers, distribución muy asimétrica
   - `ingresos`: Concentración en 51k con cola larga

---

### Pasos para el preprocesamiento

**Pre-procesamiento adicional necesario**:
1. **Limpieza de outliers**:
   - Eliminar o imputar los 3 registros con edad >120
   - Tratar el ingreso ficticio de 666,666
   - Evaluar winsorización para `ticket_promedio` (5.4% outliers)

2. **Reducción de dimensionalidad**:
   - Eliminar `total_dependientes` (r=1.00 con `tamano_hogar`)
   - Considerar PCA o selección de features para resolver multicolinealidad moderada

3. **Transformaciones**:
   - Aplicar `log1p()` a variables con sesgo fuerte: `gasto_total`, `ticket_promedio`, `ingresos`
   - Normalizar/estandarizar variables numéricas para algoritmos sensibles a escala

4. **Ingeniería de features adicional**:
   - Crear variable binaria `tiene_pareja` (Casado/Unión_Libre vs resto)
   - Codificar `educacion` ordinalmente (Básica=1, Secundaria=2, ..., Doctorado=5)
   - Crear interacciones: `educacion × estado_civil`, `gasto_total × recencia`


## Paso 1.2: Limpieza de outliers

#### 1.2.1 Metodología tratamiento de outliers   



Tal y como se vio en el preprocesamiento, los outliers representan un % muy pequeño de las muestras, por lo que se decide eliminarlos en lugar de recurrir a imputación.

In [4]:
# 1.2.2 Edad
indices = df[df["edad"] > 120].index

display(indices) # Índices de las filas a eliminar
display(df.iloc[indices]) # Mostrar filas eliminadas
df = df.drop(indices) # Eliminar filas del dataframe

Index([841, 1020, 1969], dtype='int64')

,anio_nacimiento,educacion,estado_civil,ingresos,hijos_casa,adolescentes_casa,fecha_cliente,recencia,gasto_vinos,gasto_frutas,gasto_carnes,gasto_pescado,gasto_dulces,gasto_oro,num_compras_oferta,num_compras_web,num_compras_catalogo,num_compras_tienda,num_visitas_web_mes,acepta_cmp3,acepta_cmp4,acepta_cmp5,acepta_cmp1,acepta_cmp2,reclama,respuesta,usuario_alta_datos,edad,antiguedad_dias,antiguedad_anios,gasto_total,gasto_promedio,prop_gasto_vinos,prop_gasto_frutas,prop_gasto_carnes,prop_gasto_pescado,prop_gasto_dulces,prop_gasto_oro,categorias_compradas,compras_totales,compras_online,compras_offline,tasa_compra_online,tasa_compra_oferta,ticket_promedio,tamano_hogar,total_dependientes,tiene_dependientes,hogar_unipersonal
841,1893,Secundaria,Soltero,60182.0,0,1,2014-05-17,23,8,0,5,7,0,2,1,1,0,2,4,0,0,0,0,0,0,0,us_direccion_2,132,4181,11.4,22,3.67,0.364,0.000,0.227,0.318,0.000,0.091,4,4,1,2,0.250,0.250,5.50,2,1,1,0
1020,1899,Doctorado,Union_Libre,83532.0,0,0,2013-09-26,36,755,144,562,104,64,224,1,4,6,4,1,0,0,1,0,0,0,0,admin,126,4414,12.1,1853,308.83,0.407,0.078,0.303,0.056,0.035,0.121,6,15,10,4,0.667,0.067,123.53,1,0,0,1
1969,1900,Secundaria,Divorciado,36640.0,1,0,2013-09-26,99,15,6,8,7,4,25,1,2,1,2,5,0,0,0,0,0,1,0,admin,125,4414,12.1,65,10.83,0.231,0.092,0.123,0.108,0.062,0.385,6,6,3,2,0.500,0.167,10.83,2,1,1,0


### 1.2.2 Interpretación de la limpieza de outliers

En esta sección hemos aplicado un criterio muy conservador de limpieza:

- **Edad**: se eliminaron únicamente 3 clientes con valores biológicamente imposibles (`edad > 120`), que representan una fracción mínima del conjunto de datos.
- **Ingresos**: se eliminó 1 cliente con un ingreso claramente ficticio (`ingresos = 666,666`), manteniendo el resto de valores extremos como posibles clientes de alto poder adquisitivo.

Con esta decisión:

- **Reducimos ruido evidente** sin alterar la estructura general de las distribuciones ni el balance de la variable objetivo.
- **Mantenemos prácticamente todo el tamaño muestral**, de modo que el modelo siga viendo la diversidad real de perfiles, pero sin registros manifiestamente erróneos.



In [5]:
# 1.2.3 Ingreso
indice = df[df["ingresos"]==666666].index

display(indice) # Índices de la fila a eliminar
display(df.iloc[indice]) # Mostrar fila eliminada
df = df.drop(indice) # Eliminar fila del dataframe

Index([460], dtype='int64')

,anio_nacimiento,educacion,estado_civil,ingresos,hijos_casa,adolescentes_casa,fecha_cliente,recencia,gasto_vinos,gasto_frutas,gasto_carnes,gasto_pescado,gasto_dulces,gasto_oro,num_compras_oferta,num_compras_web,num_compras_catalogo,num_compras_tienda,num_visitas_web_mes,acepta_cmp3,acepta_cmp4,acepta_cmp5,acepta_cmp1,acepta_cmp2,reclama,respuesta,usuario_alta_datos,edad,antiguedad_dias,antiguedad_anios,gasto_total,gasto_promedio,prop_gasto_vinos,prop_gasto_frutas,prop_gasto_carnes,prop_gasto_pescado,prop_gasto_dulces,prop_gasto_oro,categorias_compradas,compras_totales,compras_online,compras_offline,tasa_compra_online,tasa_compra_oferta,ticket_promedio,tamano_hogar,total_dependientes,tiene_dependientes,hogar_unipersonal
460,1977,Universitaria,Union_Libre,666666.0,1,0,2013-06-02,23,9,14,18,8,1,12,4,3,1,3,6,0,0,0,0,0,0,0,us_direccion_2,48,4530,12.4,62,10.33,0.145,0.226,0.29,0.129,0.016,0.194,6,11,4,3,0.364,0.364,5.64,2,1,1,0


#### 1.2.4 Winsorización de `ticket_promedio` 



Se decide no winsorizar ya que más adelante se aplicará transformación log1p() y se escalarán los datos

## Paso 1.3: Multicolinealidad

### 1.2.4 Interpretación de la limpieza de outliers

En esta etapa se ha aplicado un criterio conservador:

- **Edad**: se eliminaron 3 clientes con edades biológicamente imposibles (`edad > 120`), que suponen una fracción mínima del conjunto.
- **Ingresos**: se eliminó 1 cliente con un ingreso claramente ficticio (`ingresos = 666,666`), manteniendo el resto de valores altos como posibles clientes de alto poder adquisitivo.

Con ello, el dataset:

- **Reduce ruido evidente** sin modificar la forma global de las distribuciones.
- **Conserva prácticamente todas las observaciones**, manteniendo la representatividad de los perfiles reales de clientes para el modelado posterior.



In [6]:
# 1.3.1 Variables perfectamente correlacionadas
# `total_dependientes` y `tamaño_hogar` tienen un coeficiente de correlación igual a 1, por lo que eliminamos una de ellas
df = df.drop(columns=['total_dependientes']) # Eliminamos la columna de `total_dependientes`

### 1.3.2 Interpretación de `ratio_compras_online`

En esta parte del preprocesamiento:

- **Creamos la variable `ratio_compras_online`**, que mide la proporción de compras realizadas en canales online (`compras_online / compras_totales`), acotada teóricamente entre 0 y 1.
- Detectamos **3 clientes con `compras_totales = 0`**, para los que el ratio no está definido (aparecen como `NaN`). Se eliminan del dataset porque no aportan historial de compra útil para el modelado.

El efecto práctico es:

- **Mantener un ratio bien definido para todos los clientes restantes**, sin divisiones problemáticas.
- **Reducir muy ligeramente el número de observaciones**, afectando solo a clientes sin actividad, lo que no compromete la representatividad del conjunto para el problema de respuesta a campaña.



In [7]:
# 1.3.2 Alta correlación entre variables de `compras_online` y `compras_totales`
df['ratio_compras_online'] = df['compras_online'] / df['compras_totales']  # Crear ratio de compras online frente al total de compras

# Eliminamos la variable de `compras_online`
df = df.drop(columns=['compras_online'])

# Comprobamos nulos e infinitos
# Contar nulos
n_nulos = df['ratio_compras_online'].isnull().sum()
print(f"Nulos en 'ratio_compras_online': {n_nulos}")

# Contar infinitos
n_inf = np.isinf(df['ratio_compras_online']).sum()
print(f"Infinitos en 'ratio_compras_online': {n_inf}")

# Reemplazar infinitos por NaN para facilitar tratamiento
df['ratio_compras_online'] = df['ratio_compras_online'].replace([np.inf, -np.inf], np.nan)

# Filas con ratio no definido (compras_totales = 0)
indices = df[df['ratio_compras_online'].isnull()].index
display(indices)
display(df.loc[indices])

# Eliminamos esos clientes sin historial de compra
df = df.drop(index=indices)

Nulos en 'ratio_compras_online': 3
Infinitos en 'ratio_compras_online': 0


Index([1376, 1874, 1923], dtype='int64')

,anio_nacimiento,educacion,estado_civil,ingresos,hijos_casa,adolescentes_casa,fecha_cliente,recencia,gasto_vinos,gasto_frutas,gasto_carnes,gasto_pescado,gasto_dulces,gasto_oro,num_compras_oferta,num_compras_web,num_compras_catalogo,num_compras_tienda,num_visitas_web_mes,acepta_cmp3,acepta_cmp4,acepta_cmp5,acepta_cmp1,acepta_cmp2,reclama,respuesta,usuario_alta_datos,edad,antiguedad_dias,antiguedad_anios,gasto_total,gasto_promedio,prop_gasto_vinos,prop_gasto_frutas,prop_gasto_carnes,prop_gasto_pescado,prop_gasto_dulces,prop_gasto_oro,categorias_compradas,compras_totales,compras_offline,tasa_compra_online,tasa_compra_oferta,ticket_promedio,tamano_hogar,tiene_dependientes,hogar_unipersonal,ratio_compras_online
1376,1973,Universitaria,Soltero,3502.0,1,0,2013-04-13,56,2,1,1,0,0,1,0,0,0,0,14,0,0,0,0,0,0,0,admin,52,4580,12.5,5,0.83,0.400,0.200,0.200,0.000,0.000,0.200,4,0,0,0.0,0.0,0.0,2,1,0,NaN
1874,1965,Universitaria,Divorciado,4861.0,0,0,2014-06-22,20,2,1,1,1,0,1,0,0,0,0,14,0,0,0,0,0,0,0,juan.perez,60,4145,11.3,6,1.00,0.333,0.167,0.167,0.167,0.000,0.167,5,0,0,0.0,0.0,0.0,1,0,1,NaN
1923,1975,Universitaria,Divorciado,153924.0,0,0,2014-02-07,81,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,us_direccion_1,50,4280,11.7,6,1.00,0.167,0.167,0.167,0.167,0.167,0.167,6,0,0,0.0,0.0,0.0,1,0,1,NaN


## Paso 1.4: Distribuciones asimétricas

### 1.3.3 Interpretación de `ratio_compras_online`

En esta parte del preprocesamiento:

- Se crea **`ratio_compras_online`** como la proporción de compras realizadas en canales online (`compras_online / compras_totales`), lo que resume de forma compacta la preferencia de canal de cada cliente.
- Se identifican y eliminan **los clientes con `compras_totales = 0`**, para los que este ratio no está definido (aparecen como `NaN`) y que, además, no aportan historial de compra útil para predecir respuesta a campañas.

El efecto neto es:

- Mantener un ratio bien definido para todos los clientes restantes, sin divisiones problemáticas.
- Perder solo unas pocas observaciones, concentradas en clientes sin actividad, lo que no altera de forma relevante la estructura del dataset ni el balance de la variable objetivo.



In [8]:

print(f"\nDimensiones finales: {df.shape[0]} observaciones × {df.shape[1]} variables")
print(f"Valores nulos: {df.isnull().sum().sum()}")
print(f"Valores infinitos: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")

print(f"\nDistribución de tipos de datos:")
print(df.dtypes.value_counts())

print(f"\nResumen de transformaciones:")
print(f"  - Registros eliminados (outliers): {1989 - df.shape[0]}")




Dimensiones finales: 1982 observaciones × 48 variables
Valores nulos: 0
Valores infinitos: 0

Distribución de tipos de datos:
int64      31
float64    13
object      4
Name: count, dtype: int64

Resumen de transformaciones:
  - Registros eliminados (outliers): 7


#### Resumen de transformaciones realizadas:
- Variables eliminadas: 4 (total_dependientes, compras_online, adolescentes_casa, anio_nacimiento)
- Variables escaladas: Todas las numéricas
- Variables creadas: 4 (tiene_pareja, ratio_compras_online, educacion_x_estado, gasto_x_recencia)
- Sin valores nulos después de imputación 
- Sin duplicados de registros
- Sin columnas constantes (eliminadas 2 post-imputación: coste_contacto e ingresos_contacto)
- Outliers críticos detectados: Edades imposibles (132, 126, 125 años), Ingresos ficticios (666,666)

In [9]:
# 1.4.1 Transformación de variables con sesgo fuerte  
for col in ['gasto_total', 'ticket_promedio', 'ingresos']:
    df[col] = np.log1p(df[col]) # Le aplicamos a cada una de estas columna log1p

## Paso 1.5: Ingeniería de características

In [10]:
# 1.5.1 Variable binaria `tiene_pareja`
df['tiene_pareja'] = df['estado_civil'].isin(['Casado', 'Union_Libre']).astype(int)

In [11]:
# 1.5.2 Codificación ordinal de `educacion`
educ_map = {'Basica': 1, 'Secundaria': 2, 'Universitaria': 3, 'Master': 4, 'Doctorado': 5}
df['educacion'] = df['educacion'].map(educ_map)

In [12]:
# 1.5.3 Variables de interacción
df['educacion_x_estado'] = df['educacion'] * df['tiene_pareja'] # Variable de educación por estado
df['gasto_x_recencia'] = df['gasto_total'] * df['recencia'] # Variable de gasto por recencia

In [13]:
# 1.5.4 Sumar `adolescentes_casa` a `hijos_casa`
df['hijos_casa'] = df['hijos_casa'] + df['adolescentes_casa']

df = df.drop(columns=['adolescentes_casa'])

### Resumen de transformaciones aplicadas

El dataset ha sido preprocesado siguiendo las recomendaciones del EDA:

**Limpieza de outliers:**
- Eliminados 3 registros con edad >120 años (errores de captura)
- Eliminado 1 registro con ingreso ficticio (666,666)

**Reducción de multicolinealidad:**
- Eliminada variable `total_dependientes` (correlación perfecta con `tamano_hogar`)
- Creada variable `ratio_compras_online` y eliminada `compras_online` (alta correlación con `compras_totales`)
- Eliminados 3 registros con `compras_totales = 0` que generaban división por cero

**Transformaciones de distribuciones:**
- Aplicada transformación `log1p()` a: `gasto_total`, `ticket_promedio`, `ingresos`

**Ingeniería de características:**
- Creada variable binaria `tiene_pareja`
- Codificada `educacion` de manera ordinal (1-5)
- Creadas variables de interacción: `educacion_x_estado`, `gasto_x_recencia`
- Consolidada `adolescentes_casa` en `hijos_casa`
- Eliminada variable redundante `anio_nacimiento`

El dataset preprocesado está listo para modelado predictivo y se exporta a `data/processed/supermercado_preprocesado.csv`.


In [14]:
# 1.5.5 Eliminar variable `anio_nacimiento`
# Como ya tenemos variable `edad`, `anio_nacimiento` se vuelve redundante
df = df.drop(columns=['anio_nacimiento'])

In [15]:
# 1.6.1 Verificación final del dataset preprocesado
print(f"Dimensiones finales: {df.shape[0]} observaciones × {df.shape[1]} variables")
print(f"Valores nulos totales: {df.isnull().sum().sum()}")
print(f"Valores infinitos totales: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")

print("\nDistribución de tipos de datos:")
print(df.dtypes.value_counts())


Dimensiones finales: 1982 observaciones × 49 variables
Valores nulos totales: 0
Valores infinitos totales: 0

Distribución de tipos de datos:
int64      31
float64    15
object      3
Name: count, dtype: int64


### Interpretación de la verificación final

Con esta verificación confirmamos que, antes de exportar:

- **Las dimensiones finales** reflejan solo la eliminación de unos pocos registros atípicos y de clientes sin compras, manteniendo prácticamente todo el conjunto original.
- **No quedan valores nulos ni infinitos** en las variables numéricas, lo que evita problemas silenciosos en la fase de modelado.
- **La distribución de tipos de datos** es coherente con el uso previsto: variables numéricas (incluidas las transformadas y escaladas) listas para los algoritmos, y variables categóricas en formato adecuado para su codificación posterior.

A partir de aquí, `supermercado_preprocesado.csv` se puede utilizar directamente en los notebooks de modelado sin pasos adicionales de limpieza básica.



In [16]:
# Verificación final del dataset preprocesado
print(f"Dimensiones finales: {df.shape[0]} observaciones × {df.shape[1]} variables")
print(f"Valores nulos totales: {df.isnull().sum().sum()}")
print(f"Valores infinitos totales: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")

print("\nDistribución de tipos de datos:")
print(df.dtypes.value_counts())


Dimensiones finales: 1982 observaciones × 49 variables
Valores nulos totales: 0
Valores infinitos totales: 0

Distribución de tipos de datos:
int64      31
float64    15
object      3
Name: count, dtype: int64


### 1.6.1 Interpretación de la verificación final

Antes de exportar el dataset confirmamos que:

- **Las dimensiones finales** reflejan únicamente la eliminación de unos pocos registros atípicos y de clientes sin compras, manteniendo prácticamente todo el conjunto original.
- **No quedan valores nulos ni infinitos** en las variables numéricas, lo que evita errores silenciosos en los modelos.
- **Los tipos de datos son coherentes** (numéricos escalados para modelos, categóricos en formato adecuado para su codificación posterior).

Con estas comprobaciones, el archivo `supermercado_preprocesado.csv` queda listo para ser utilizado en la fase de modelado sin pasos adicionales de limpieza básica.



## Paso 1.6: Exportar el dataset con los datos procesados

In [17]:
# Exportar dataset con los datos procesados
OUTPUT_FILE_FEATURES = "data/processed/supermercado_preprocesado.csv"
df.to_csv(OUTPUT_FILE_FEATURES, index=False)


## Paso 1.7: Preprocesado para clusterización

Los modelos usados para clusterización suelen usar distancias y funcionan mal con datos de alta dimensionalidad, por lo que necesitamos escalar los datos y realizar un PCA que nos permita reducir la dimensionalidad.

In [18]:
# 1.7.1 Escalar los datos

# Columnas numéricas
cols_num = df.select_dtypes(include=["int64", "float64"]).columns

# Columnas binarias (solo 0 y 1)
binarias = [c for c in cols_num if set(df[c].dropna().unique()).issubset({0, 1})]

# Columnas numéricas que SÍ se escalarán
a_escalar = [c for c in cols_num if c not in binarias]

# 4. Escalar
scaler = StandardScaler()
df[a_escalar] = scaler.fit_transform(df[a_escalar])

In [19]:
# 1.7.2 PCA
pca = PCA(n_components=0.80)
X_pca = pca.fit_transform(df[a_escalar])


In [20]:
n_componentes = pca.n_components_
print("Número de componentes retenidas:", n_componentes)

componentes = pd.DataFrame(
    pca.components_,
    columns=a_escalar,
    index=[f'PC{i+1}' for i in range(pca.n_components_)]
)
print(componentes)

componentes_abs = componentes.abs()
componentes_abs.idxmax(axis=1)  # Variable más influyente por componente


Número de componentes retenidas: 10
      educacion  ingresos  hijos_casa  recencia  gasto_vinos  gasto_frutas  gasto_carnes  gasto_pescado  gasto_dulces  gasto_oro  num_compras_oferta  \
PC1    0.032038  0.220703   -0.157241  0.015983     0.228463      0.182989      0.222765       0.190762      0.183147   0.157335           -0.027720   
PC2    0.267009  0.127677    0.223576  0.004925     0.144785     -0.150919     -0.077629      -0.154111     -0.149058  -0.049384            0.190070   
PC3   -0.163100 -0.127696    0.151415  0.039680     0.028238      0.016204     -0.068713       0.021754      0.031087   0.167244            0.374365   
PC4    0.003229 -0.006670    0.051619  0.721780    -0.039369     -0.030480      0.050230      -0.021562     -0.024678  -0.022019           -0.005364   
PC5    0.047964 -0.092705   -0.174740 -0.014310     0.013624     -0.146731     -0.104018      -0.132702     -0.103063   0.163300           -0.200067   
PC6    0.139692 -0.043860   -0.259178 -0.005211     

PC1            gasto_total
PC2       prop_gasto_vinos
PC3       antiguedad_anios
PC4               recencia
PC5     tasa_compra_online
PC6        antiguedad_dias
PC7      prop_gasto_carnes
PC8     educacion_x_estado
PC9                   edad
PC10    educacion_x_estado
dtype: object

### Resumen de transformaciones aplicadas

El dataset ha sido preprocesado siguiendo las recomendaciones del EDA:


**Transformaciones de distribuciones:**`
- Escaladas todas las variables numéricas con `StandardScaler`

**Reducción de dimensionalidad**
- PCA

El dataset preprocesado está listo para modelado de clusterización y se exporta a `data/processed/supermercado_cluster.csv`.

In [21]:
# 1.7.3 Exportar dataset
#OUTPUT_FILE_FEATURES = "data/processed/supermercado_cluster.csv"
#df.to_csv(OUTPUT_FILE_FEATURES, index=False)
